In [55]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [56]:
#!/bin/bash
! kaggle datasets download tanujshriyan/yelp-polarity-reviews

Dataset URL: https://www.kaggle.com/datasets/tanujshriyan/yelp-polarity-reviews
License(s): unknown
yelp-polarity-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [57]:
import zipfile
zip_ref = zipfile.ZipFile('/content/yelp-polarity-reviews.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [58]:
import pandas as pd

In [59]:
df = pd.read_csv('/content/train.csv' , header = None)

In [60]:
df.shape

(650000, 2)

In [61]:
df.head()

,0,1
0,5,dr. goldberg offers everything i look for in a...
1,2,"Unfortunately, the frustration of being Dr. Go..."
2,4,Been going to Dr. Goldberg for over 10 years. ...
3,4,Got a letter in the mail last week that said D...
4,1,I don't know what Dr. Goldberg was like before...


In [62]:
df.dropna(inplace = True)

In [63]:
df.shape

(650000, 2)

In [64]:
df.columns = ['sentiment' , 'review']

In [65]:
df = df.sample(25000 , random_state = 42).reset_index(drop = True)

In [66]:
df.shape

(25000, 2)

In [67]:
df.head()

,sentiment,review
0,1,"First of all i'm not a big fan of buffet, i tr..."
1,2,Thanks Yelp. I was looking for the words to de...
2,3,Service was so-so. They were receiving a deliv...
3,3,Stamoolis Brothers is one of the Strip Distric...
4,1,I want to give a 2 stars because the service s...


In [68]:
test_df = pd.read_csv('/content/test.csv' , header = None)

In [69]:
test_df.shape

(50000, 2)

In [70]:
test_df.dropna(inplace = True)

In [71]:
test_df.head()

,0,1
0,1,I got 'new' tires from them and within two wee...
1,1,Don't waste your time. We had two different p...
2,1,All I can say is the worst! We were the only 2...
3,1,I have been to this restaurant twice and was d...
4,1,Food was NOT GOOD at all! My husband & I ate h...


In [72]:
test_df.columns = ['sentiment' , 'review']

In [73]:
test_df.head()

,sentiment,review
0,1,I got 'new' tires from them and within two wee...
1,1,Don't waste your time. We had two different p...
2,1,All I can say is the worst! We were the only 2...
3,1,I have been to this restaurant twice and was d...
4,1,Food was NOT GOOD at all! My husband & I ate h...


In [74]:
test_df = test_df.sample(n = 10000 , random_state = 42).reset_index(drop = True)

In [75]:
test_df.shape

(10000, 2)

In [76]:
import re
import string
import nltk
! pip install contractions

import contractions

In [77]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [78]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [79]:
stop_word = stopwords.words('english')

lemma = WordNetLemmatizer()

In [80]:
def clean_text(text):

  text = contractions.fix(text)
  text = text.lower()
  text = re.sub(r'http\S+' ,  '', text)
  text = re.sub(r'<.*?>' ,'' , text)
  text = text.translate(str.maketrans('' ,'' , string.punctuation))
  token = word_tokenize(text)
  final = [lemma.lemmatize(word) for word in token if word not in stop_word]
  return  ' '.join(final)

In [81]:
df['review'] = df['review'].apply(clean_text)

In [82]:
test_df['review']  = test_df['review'].apply(clean_text)

In [83]:
df.head()

,sentiment,review
0,1,first big fan buffet tried got 50 credit stayi...
1,2,thanks yelp looking word describe place meh se...
2,3,service soso receiving delivery might food hot...
3,3,stamoolis brother one strip district storefron...
4,1,want give 2 star service staff friendly good w...


In [84]:
test_df.head()

,sentiment,review
0,5,came day ago lease sure size needed guessed th...
1,1,chose 4 queen visit la vega several reason adm...
2,4,went day wedding town last minute pedicure rea...
3,2,strange little thing sour experience good time...
4,5,visit several time year food always fresh well...


In [85]:
df['sentiment'].value_counts()

,count
sentiment,
1,5057
4,5046
3,5024
2,4949
5,4924


In [86]:
from sklearn.preprocessing import LabelEncoder

In [87]:
le = LabelEncoder()

In [88]:
df['sentiment'] = le.fit_transform(df['sentiment'])

In [89]:
test_df['sentiment'] = le.transform(test_df['sentiment'])

In [90]:
x_train = df['review']
y_train = df['sentiment']

In [91]:
x_test = test_df['review']
y_test = test_df['sentiment']

In [92]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [93]:
token = Tokenizer(num_words = 15000 , oov_token = 'OOV')

In [94]:
token.fit_on_texts(x_train)

In [95]:
train_seq = token.texts_to_sequences(x_train)

In [96]:
test_seq = token.texts_to_sequences(x_test)

In [97]:
max_len = max([len(seq) for seq in train_seq])
max_len

503

In [98]:
train_pad = pad_sequences(train_seq , padding ='post', maxlen = max_len)

In [99]:
test_pad = pad_sequences(test_seq ,  padding = 'post' , maxlen = max_len)

In [100]:
from keras.models import *
from keras.layers import *

In [101]:
y_train.value_counts()

,count
sentiment,
0,5057
3,5046
2,5024
1,4949
4,4924


In [102]:
input = Input(shape=(max_len,))

emb = Embedding(input_dim=15000, output_dim=32, mask_zero=True)(input)

d1 = Bidirectional(GRU(64))(emb)
d2 = Dropout(0.5)(d1)
d3 = Dense(64, activation='relu')(d2)

output = Dense(5, activation='softmax')(d3)

model = Model(inputs = input, outputs = output)


In [103]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 503)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 503, 32)   │    480,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 503)       │          0 │ input_layer_1[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 128)       │     37,632 │ embedding_1[0][0… │
│ (Bidirectional)     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 5)         │        325 │ dense_2[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 526,213 (2.01 MB)

 Trainable params: 526,213 (2.01 MB)

 Non-trainable params: 0 (0.00 B)

In [104]:
from keras.optimizers import *
from keras.losses import *
from keras.callbacks import *

In [105]:
model.compile(optimizer = Adam(3e-4) , loss = SparseCategoricalCrossentropy(from_logits = False) , metrics = ['accuracy'])

In [106]:
es = EarlyStopping(monitor = 'val_loss' , restore_best_weights = True , patience = 3 , mode = 'min')

In [107]:
ls = ReduceLROnPlateau(monitor = 'val_loss', factor = 0.4 , patience = 3)

In [108]:
model.fit(train_pad ,y_train , validation_data = (test_pad , y_test) , epochs = 30 , batch_size = 16 , verbose = 1 )

Epoch 1/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 36s 22ms/step - accuracy: 0.2942 - loss: 1.4892 - val_accuracy: 0.5226 - val_loss: 1.0874
Epoch 2/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - accuracy: 0.5768 - loss: 0.9887 - val_accuracy: 0.5361 - val_loss: 1.0702
Epoch 3/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 34s 22ms/step - accuracy: 0.6435 - loss: 0.8528 - val_accuracy: 0.5356 - val_loss: 1.1012
Epoch 4/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.6850 - loss: 0.7775 - val_accuracy: 0.5237 - val_loss: 1.1595
Epoch 5/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 34s 22ms/step - accuracy: 0.7240 - loss: 0.6924 - val_accuracy: 0.5156 - val_loss: 1.2312
Epoch 6/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - accuracy: 0.7556 - loss: 0.6298 - val_accuracy: 0.5085 - val_loss: 1.3304
Epoch 7/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 34s 22ms/step - accuracy: 0.7856 - loss: 0.5632 - val_accuracy: 0.5023 - val_loss: 1.4383
Epoch 8/30
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - accuracy: 0.8204 -

In [109]:
 model.save('day_10_yelp_polarity.keras')

In [111]:
import joblib

joblib.dump(token, 'day_10_yelp_polarity_token.pkl')

['day_10_yelp_polarity_token.pkl']